In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
! hdfs dfs -ls /data/wiki

Found 2 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part
drwxr-xr-x   - jovyan supergroup          0 2018-05-13 18:46 /data/wiki/gaio_files


In [5]:
%ls

20180513_save_orig_demo.ipynb  mapper.py  reducer.py       supervisord.pid
DemoNotebook.ipynb             README.md  supervisord.log  test_article.txt


In [40]:
!hdfs dfs -copyFromLocal a.txt /data/wiki/gaio_files

In [44]:
! hdfs dfs -ls /data/wiki/gaio_files

Found 1 items
-rw-r--r--   1 jovyan supergroup     849050 2018-05-13 19:39 /data/wiki/gaio_files/a.txt


In [42]:
! hdfs dfs -ls /data/wiki/en_articles_part

Found 1 items
-rwxrwxrwx   1 jovyan supergroup   76861985 2017-10-17 13:15 /data/wiki/en_articles_part/articles-part


In [45]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapred.jab.name="Streaming wordCount" \
    -files mapper.py \
    -mapper "python mapper.py" \
    -numReduceTasks 0 \
    -input /data/wiki/gaio_files \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

anarchism	1
anarchism	1
is	1
often	1
defined	1
as	1
a	1
political	1
philosophy	1
which	1


rm: `wordcount_result_1526240480398252': No such file or directory
18/05/13 19:41:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/13 19:41:26 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/13 19:41:27 INFO mapred.FileInputFormat: Total input files to process : 1
18/05/13 19:41:27 INFO mapreduce.JobSubmitter: number of splits:2
18/05/13 19:41:27 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1526235386425_0007
18/05/13 19:41:27 INFO impl.YarnClientImpl: Submitted application application_1526235386425_0007
18/05/13 19:41:27 INFO mapreduce.Job: The url to track the job: http://fabdd7fdded6:8088/proxy/application_1526235386425_0007/
18/05/13 19:41:27 INFO mapreduce.Job: Running job: job_1526235386425_0007
18/05/13 19:41:36 INFO mapreduce.Job: Job job_1526235386425_0007 running in uber mode : false
18/05/13 19:41:36 INFO mapreduce.Job:  map 0% reduce 0%
18/05/13 19:41:53 INFO mapreduce.Job:  map 100% reduce 0%
18/05/13 19:

In [46]:
!hdfs dfs -ls wordcount_result_1526240480398252

Found 3 items
-rw-r--r--   1 jovyan supergroup          0 2018-05-13 19:41 wordcount_result_1526240480398252/_SUCCESS
-rw-r--r--   1 jovyan supergroup     630534 2018-05-13 19:41 wordcount_result_1526240480398252/part-00000
-rw-r--r--   1 jovyan supergroup     449970 2018-05-13 19:41 wordcount_result_1526240480398252/part-00001


In [47]:
!hdfs dfs -cat wordcount_result_1526240480398252/part-00000 

anarchism	1
anarchism	1
is	1
often	1
defined	1
as	1
a	1
political	1
philosophy	1
which	1
holds	1
the	1
state	1
to	1
be	1
undesirable	1
unnecessary	1
or	1
harmful	1
the	1
following	1
sources	1
cite	1
anarchism	1
as	1
a	1
political	1
philosophy	1
slevin	1
carl	1
anarchism	1
the	1
concise	1
oxford	1
dictionary	1
of	1
politics	1
ed	1
iain	1
mclean	1
and	1
alistair	1
mcmillan	1
oxford	1
university	1
press	1
2003	1
however	1
others	1
argue	1
that	1
while	1
anti-statism	1
is	1
central	1
it	1
is	1
inadequate	1
to	1
define	1
anarchism	1
therefore	1
they	1
argue	1
instead	1
that	1
anarchism	1
entails	1
opposing	1
authority	1
or	1
hierarchical	1
organization	1
in	1
the	1
conduct	1
of	1
human	1
relations	1
including	1
but	1
not	1
only	1
the	1
state	1
system	1
proponents	1
of	1
anarchism	1
known	1
as	1
anarchists	1
advocate	1
stateless	1
societies	1
based	1
on	1
non-hierarchical	1
free	1
associations	1
as	1
a	1
subtle	1
and	1
anti-dogmatic	1
philosophy	1
anarchism	1
draws	1
on	1
many	1
currents	1
o

the	1
19th	1
century	1
ecole	1
polytechnique	1
centre	1
de	1
recherce	1
en	1
epistemologie	1
appliquee	1
unité	1
associée	1
au	1
cnrs	1
2004	1
heider	1
ulrike	1
anarchism:left	1
right	1
and	1
green	1
city	1
lights	1
1994	1
p	1
3	1
outhwaite	1
william	1
the	1
blackwell	1
dictionary	1
of	1
modern	1
social	1
thought	1
anarchism	1
entry	1
p	1
21	1
2002	1
bottomore	1
tom	1
dictionary	1
of	1
marxist	1
thought	1
anarchism	1
entry	1
1991	1
ostergaard	1
geofrey	1
resisting	1
the	1
nation	1
state	1
the	1
anarchist	1
and	1
pacifist	1
tradition	1
anarchism	1
as	1
a	1
tradition	1
of	1
political	1
thought	1
peace	1
pledge	1
union	1
publications	1
drawing	1
from	1
austrian	1
school	1
economics	1
study	1
of	1
law	1
and	1
economics	1
and	1
public	1
choice	1
theory	1
there	1
is	1
a	1
strong	1
current	1
within	1
anarchism	1
which	1
does	1
not	1
consider	1
that	1
anarcho-capitalism	1
can	1
be	1
considered	1
a	1
part	1
of	1
the	1
anarchist	1
movement	1
due	1
to	1
the	1
fact	1
that	1
anarchism	1
has	1
histo

us	1
study	1
found	1
a	1
14	1
average	1
loss	1
of	1
annual	1
income	1
in	1
families	1
of	1
children	1
with	1
asd	1
and	1
a	1
related	1
study	1
found	1
that	1
asd	1
is	1
associated	1
with	1
higher	1
probability	1
that	1
child	1
care	1
problems	1
will	1
greatly	1
affect	1
parental	1
employment	1
us	1
states	1
increasingly	1
require	1
private	1
health	1
insurance	1
to	1
cover	1
autism	1
services	1
shifting	1
costs	1
from	1
publicly	1
funded	1
education	1
programs	1
to	1
privately	1
funded	1
health	1
insurance	1
after	1
childhood	1
key	1
treatment	1
issues	1
include	1
residential	1
care	1
job	1
training	1
and	1
placement	1
sexuality	1
social	1
skills	1
and	1
estate	1
planning	1
prognosisthere	1
is	1
no	1
known	1
cure	1
children	1
recover	1
occasionally	1
so	1
that	1
they	1
lose	1
their	1
diagnosis	1
of	1
asd	1
this	1
occurs	1
sometimes	1
after	1
intensive	1
treatment	1
and	1
sometimes	1
not	1
it	1
is	1
not	1
known	1
how	1
often	1
recovery	1
happens	1
reported	1
rates	1
in	1
unselected	1
sa

from	1
lightning	1
and	1
ninth	1
in	1
the	1
number	1
of	1
deaths	1
from	1
lightning	1
strikes	1
per	1
capita	1
alabama	1
along	1
with	1
kansas	1
has	1
the	1
most	1
reported	1
ef5	1
tornadoes	1
of	1
any	1
state	1
according	1
to	1
statistics	1
from	1
the	1
national	1
climatic	1
data	1
center	1
for	1
the	1
period	1
january	1
1	1
1950	1
to	1
october	1
31	1
2006	1
several	1
long-tracked	1
f5	1
tornadoes	1
have	1
contributed	1
to	1
alabama	1
reporting	1
more	1
tornado	1
fatalities	1
than	1
any	1
other	1
state	1
even	1
surpassing	1
texas	1
which	1
has	1
a	1
much	1
larger	1
area	1
within	1
tornado	1
alley	1
the	1
state	1
suffered	1
tremendous	1
damage	1
in	1
the	1
super	1
outbreak	1
of	1
april	1
1974	1
and	1
the	1
april	1
25–28	1
2011	1
tornado	1
outbreak	1
the	1
outbreak	1
in	1
april	1
2011	1
produced	1
a	1
record	1
amount	1
of	1
tornadoes	1
in	1
the	1
state	1
the	1
tally	1
reached	1
62	1
the	1
peak	1
season	1
for	1
tornadoes	1
varies	1
from	1
the	1
northern	1
to	1
southern	1
parts	1
of	1
the

but	1
wish	1
to	1
get	1
them	1
from	1
the	1
god	1
of	1
the	1
island	1
himself	1
they	1
consult	1
achilles	1
oracle	1
they	1
ask	1
permission	1
to	1
slaughter	1
the	1
victims	1
chosen	1
from	1
among	1
the	1
animals	1
that	1
graze	1
freely	1
on	1
the	1
island	1
and	1
to	1
deposit	1
in	1
exchange	1
the	1
price	1
which	1
they	1
consider	1
fair	1
but	1
in	1
case	1
the	1
oracle	1
denies	1
them	1
permission	1
because	1
there	1
is	1
an	1
oracle	1
here	1
they	1
add	1
something	1
to	1
the	1
price	1
offered	1
and	1
if	1
the	1
oracle	1
refuses	1
again	1
they	1
add	1
something	1
more	1
until	1
at	1
last	1
the	1
oracle	1
agrees	1
that	1
the	1
price	1
is	1
sufficient	1
and	1
then	1
the	1
victim	1
doesn’t	1
run	1
away	1
any	1
more	1
but	1
waits	1
willingly	1
to	1
be	1
caught	1
so	1
there	1
is	1
a	1
great	1
quantity	1
of	1
silver	1
there	1
consecrated	1
to	1
the	1
hero	1
as	1
price	1
for	1
the	1
sacrificial	1
victims	1
to	1
some	1
of	1
the	1
people	1
who	1
come	1
to	1
this	1
island	1
achilles	1
appears

execution	1
of	1
lincoln's	1
order	1
to	1
meet	1
that	1
request	1
was	1
seen	1
by	1
the	1
secessionists	1
as	1
an	1
act	1
of	1
war	1
on	1
april	1
12	1
1861	1
confederate	1
forces	1
fired	1
on	1
union	1
troops	1
at	1
fort	1
sumter	1
forcing	1
them	1
to	1
surrender	1
and	1
began	1
the	1
war	1
historian	1
allan	1
nevins	1
argued	1
that	1
the	1
newly	1
inaugurated	1
lincoln	1
made	1
three	1
miscalculations	1
underestimating	1
the	1
gravity	1
of	1
the	1
crisis	1
exaggerating	1
the	1
strength	1
of	1
unionist	1
sentiment	1
in	1
the	1
south	1
and	1
not	1
realizing	1
the	1
southern	1
unionists	1
were	1
insisting	1
there	1
be	1
no	1
invasion	1
william	1
tecumseh	1
sherman	1
talked	1
to	1
lincoln	1
during	1
inauguration	1
week	1
and	1
was	1
sadly	1
disappointed	1
at	1
his	1
failure	1
to	1
realize	1
that	1
the	1
country	1
was	1
sleeping	1
on	1
a	1
volcano	1
and	1
that	1
the	1
south	1
was	1
preparing	1
for	1
war	1
donald	1
concludes	1
that	1
his	1
repeated	1
efforts	1
to	1
avoid	1
collision	1
in	1


on	1
the	1
mall	1
in	1
washington	1
in	1
the	1
new	1
deal	1
era	1
liberals	1
honored	1
lincoln	1
not	1
so	1
much	1
as	1
the	1
self-made	1
man	1
or	1
the	1
great	1
war	1
president	1
but	1
as	1
the	1
advocate	1
of	1
the	1
common	1
man	1
who	1
doubtless	1
would	1
have	1
supported	1
the	1
welfare	1
state	1
in	1
the	1
cold	1
war	1
years	1
lincoln's	1
image	1
shifted	1
to	1
emphasize	1
the	1
symbol	1
of	1
freedom	1
who	1
brought	1
hope	1
to	1
those	1
oppressed	1
by	1
communist	1
regimes	1
by	1
the	1
1970s	1
lincoln	1
had	1
become	1
a	1
hero	1
to	1
political	1
conservatives	1
for	1
his	1
intense	1
nationalism	1
support	1
for	1
business	1
his	1
insistence	1
on	1
stopping	1
the	1
spread	1
of	1
human	1
bondage	1
his	1
acting	1
in	1
terms	1
of	1
lockean	1
and	1
burkean	1
principles	1
on	1
behalf	1
of	1
both	1
liberty	1
and	1
tradition	1
and	1
his	1
devotion	1
to	1
the	1
principles	1
of	1
the	1
founding	1
fathers	1
as	1
a	1
whig	1
activist	1
lincoln	1
was	1
a	1
spokesman	1
for	1
business	1
interes

within	1
aristotle's	1
school	1
as	1
opposed	1
on	1
the	1
other	1
hand	1
to	1
the	1
dialogues	1
and	1
other	1
exoteric	1
texts	1
which	1
aristotle	1
published	1
more	1
widely	1
during	1
his	1
lifetime	1
the	1
consensus	1
is	1
that	1
andronicus	1
of	1
rhodes	1
collected	1
the	1
esoteric	1
works	1
of	1
aristotle's	1
school	1
which	1
existed	1
in	1
the	1
form	1
of	1
smaller	1
separate	1
works	1
distinguished	1
them	1
from	1
those	1
of	1
theophrastus	1
and	1
other	1
peripatetics	1
edited	1
them	1
and	1
finally	1
compiled	1
them	1
into	1
the	1
more	1
cohesive	1
larger	1
works	1
as	1
they	1
are	1
known	1
today	1
legacy	1
more	1
than	1
twenty-three	1
hundred	1
years	1
after	1
his	1
death	1
aristotle	1
remains	1
one	1
of	1
the	1
most	1
influential	1
people	1
who	1
ever	1
lived	1
he	1
contributed	1
to	1
almost	1
every	1
field	1
of	1
human	1
knowledge	1
then	1
in	1
existence	1
and	1
he	1
was	1
the	1
founder	1
of	1
many	1
new	1
fields	1
according	1
to	1
the	1
philosopher	1
bryan	1
magee	1
it	1
is

it	1
is	1
inadequate	1
to	1
define	1
anarchism	1
therefore	1
they	1
argue	1
instead	1
that	1
anarchism	1
entails	1
opposing	1
authority	1
or	1
hierarchical	1
organization	1
in	1
the	1
conduct	1
of	1
human	1
relations	1
including	1
but	1
not	1
only	1
the	1
state	1
system	1
proponents	1
of	1
anarchism	1
known	1
as	1
anarchists	1
advocate	1
stateless	1
societies	1
based	1
on	1
non-hierarchical	1
free	1
associations	1
as	1
a	1
subtle	1
and	1
anti-dogmatic	1
philosophy	1
anarchism	1
draws	1
on	1
many	1
currents	1
of	1
thought	1
and	1
strategy	1
anarchism	1
does	1
not	1
offer	1
a	1
fixed	1
body	1
of	1
doctrine	1
from	1
a	1
single	1
particular	1
world	1
view	1
instead	1
fluxing	1
and	1
flowing	1
as	1
a	1
philosophy	1
there	1
are	1
many	1
types	1
and	1
traditions	1
of	1
anarchism	1
not	1
all	1
of	1
which	1
are	1
mutually	1
exclusive	1
anarchist	1
schools	1
of	1
thought	1
can	1
differ	1
fundamentally	1
supporting	1
anything	1
from	1
extreme	1
individualism	1
to	1
complete	1
collectivism	1
strai

theory	1
and	1
poststructuralist	1
thought	1
drawing	1
from	1
diverse	1
ideas	1
including	1
post-modernism	1
autonomist	1
marxism	1
post-left	1
anarchy	1
situationism	1
and	1
postcolonialism.anarcho-capitalism	1
advocates	1
the	1
elimination	1
of	1
the	1
state	1
in	1
favor	1
of	1
individual	1
sovereignty	1
in	1
a	1
free	1
market	1
edward	1
stringham	1
anarchy	1
and	1
the	1
law	1
the	1
political	1
economy	1
of	1
choice	1
p	1
51	1
anarcho-capitalism	1
developed	1
from	1
radical	1
anti-state	1
libertarianism	1
and	1
individualist	1
anarchism	1
tormey	1
simon	1
anti-capitalism	1
one	1
world	1
2004	1
perlin	1
terry	1
m	1
contemporary	1
anarchism	1
transaction	1
books	1
nj	1
1979	1
raico	1
ralph	1
authentic	1
german	1
liberalism	1
of	1
the	1
19th	1
century	1
ecole	1
polytechnique	1
centre	1
de	1
recherce	1
en	1
epistemologie	1
appliquee	1
unité	1
associée	1
au	1
cnrs	1
2004	1
heider	1
ulrike	1
anarchism:left	1
right	1
and	1
green	1
city	1
lights	1
1994	1
p	1
3	1
outhwaite	1
william	1
the	1
b